In [6]:
# test_api.py
import requests
import json
from datetime import datetime

BASE_URL = "http://localhost:8765"
TOKEN = "0214d19adcc241b385dad8ddccbfd17a"  # Replace with actual token
HEADERS = {"Authorization": f"Bearer {TOKEN}"}

def test_submit_token():
    response = requests.post(
        f"{BASE_URL}/submit_token",
        json={"token": TOKEN}
    )
    print("Submit Token:", response.json())

def test_get_threads():
    response = requests.get(
        f"{BASE_URL}/threads",
        headers=HEADERS
    )
    print("Get Threads:", response.json())

def test_chat():
    response = requests.post(
        f"{BASE_URL}/chat",
        headers=HEADERS,
        json={
            "query": "What was discussed in my last meeting?",
            "thread_id": None,  # Optional
            "model": "gpt-4",  # Optional
            "temperature": 0.7  # Optional
        }
    )
    # Since this is a streaming response, we need to handle it differently
    for line in response.iter_lines():
        if line:
            data = json.loads(line.decode('utf-8').replace('data: ', ''))
            print("Chat Response:", data)

def test_get_meeting_timestamps():
    response = requests.get(
        f"{BASE_URL}/user/meetings/timestamps",
        headers=HEADERS
    )
    print("Meeting Timestamps:", response.json())

def test_get_meeting_details():
    meeting_id = "your_meeting_id_here"  # Replace with actual UUID
    response = requests.get(
        f"{BASE_URL}/meeting/{meeting_id}/details",
        headers=HEADERS
    )
    print("Meeting Details:", response.json())

def test_create_share_link():
    response = requests.post(
        f"{BASE_URL}/share-links",
        headers=HEADERS,
        json={
            "access_level": "READ",
            "target_email": "test@example.com",
            "expiration_hours": 24,
            "include_existing_meetings": True
        }
    )
    print("Share Link:", response.json())

In [7]:
# test_api.py
def test_get_all_meetings():
    # Basic request
    response = requests.get(
        f"{BASE_URL}/meetings/all",
        headers=HEADERS
    )
    print("All Meetings:", response.json())

    # With pagination
    response = requests.get(
        f"{BASE_URL}/meetings/all",
        headers=HEADERS,
        params={
            "offset": 0,
            "limit": 10
        }
    )
    print("Paginated Meetings:", response.json())

In [8]:
test_submit_token()

Submit Token: {'user_id': 'aa8d9413-8d77-437f-96f1-0187bc9d412f', 'user_name': 'Sergey Ryabenko'}


In [9]:
def test_get_all_meetings():
    # Test basic request
    response = requests.get(
        f"{BASE_URL}/meetings/all",
        headers=HEADERS
    )
    print("All Meetings (Basic):", response.json())

    # Test pagination
    response = requests.get(
        f"{BASE_URL}/meetings/all",
        headers=HEADERS,
        params={
            "offset": 0,
            "limit": 10
        }
    )
    print("Meetings (Paginated):", response.json())

    # Test with date filtering
    response = requests.get(
        f"{BASE_URL}/meetings/all",
        headers=HEADERS,
        params={
            "start_date": "2024-01-01",
            "end_date": datetime.now().strftime("%Y-%m-%d")
        }
    )
    print("Meetings (Date Filtered):", response.json())

    # Test sorting
    response = requests.get(
        f"{BASE_URL}/meetings/all",
        headers=HEADERS,
        params={
            "sort_by": "date",
            "sort_order": "desc"
        }
    )
    print("Meetings (Sorted):", response.json())

    # Test combined parameters
    response = requests.get(
        f"{BASE_URL}/meetings/all",
        headers=HEADERS,
        params={
            "offset": 0,
            "limit": 5,
            "start_date": "2024-01-01",
            "sort_by": "date",
            "sort_order": "desc"
        }
    )
    return response.json()

In [17]:
def test_get_meeting_details():
    # Test with valid meeting ID from previous response
    meeting_id = "37769a4e-fd8d-4193-8a91-f001a16649b2"  # Using meeting ID from your output
    response = requests.get(
        f"{BASE_URL}/meeting/{meeting_id}/details",
        headers=HEADERS
    )
    return response.json()
r = test_get_meeting_details()['transcript']

In [18]:
json.loads(r)

[{'speaker': 'Sergey Ryabenko',
  'speaker_id': 'TBD',
  'content': ' Да, привет.',
  'html_content': 'Да, привет.',
  'html_content_short': '',
  'keywords': [],
  'timestamp': '2024-11-03T12:26:49.350000Z'},
 {'speaker': 'TBD',
  'speaker_id': 'TBD',
  'content': ' Ничего вам.',
  'html_content': 'Ничего вам.',
  'html_content_short': '',
  'keywords': [],
  'timestamp': '2024-11-03T12:26:53.960000Z'},
 {'speaker': 'Dmitriy Grankin',
  'speaker_id': 'TBD',
  'content': ' Ну и что?',
  'html_content': 'Ну и что?',
  'html_content_short': '',
  'keywords': [],
  'timestamp': '2024-11-03T12:26:54.635999Z'},
 {'speaker': 'TBD',
  'speaker_id': 'TBD',
  'content': ' Ну, что я тут...',
  'html_content': 'Ну, что я тут...',
  'html_content_short': '',
  'keywords': [],
  'timestamp': '2024-11-03T12:26:58.475999Z'},
 {'speaker': 'Dmitriy Grankin',
  'speaker_id': 'TBD',
  'content': ' Такой бэк довольно большой построил с фронтом. Так, что надо сделать?',
  'html_content': 'Такой <b>бэк</b> 

In [14]:
def test_search_transcripts():

    response = requests.post(
        f"{BASE_URL}/search/transcripts",
        headers=HEADERS,
        json={
            "query": "API discussion",
            "meeting_ids": ["37769a4e-fd8d-4193-8a91-f001a16649b2"],  # Use actual meeting ID
            "min_score": 0.0001
        }
    )
    return response.json()

# Run the test
test_search_transcripts()

{'results': [{'content': '<b> Да, привет.</b>',
   'speaker': 'Sergey Ryabenko',
   'timestamp': '2024-11-03T12:26:49.350000Z',
   'meeting_id': '37769a4e-fd8d-4193-8a91-f001a16649b2'},
  {'content': '<b> Ничего вам.</b>',
   'speaker': 'TBD',
   'timestamp': '2024-11-03T12:26:53.960000Z',
   'meeting_id': '37769a4e-fd8d-4193-8a91-f001a16649b2'},
  {'content': '<b> Ну и что?</b>',
   'speaker': 'Dmitriy Grankin',
   'timestamp': '2024-11-03T12:26:54.635999Z',
   'meeting_id': '37769a4e-fd8d-4193-8a91-f001a16649b2'},
  {'content': '<b> Ну, что я тут...</b>',
   'speaker': 'TBD',
   'timestamp': '2024-11-03T12:26:58.475999Z',
   'meeting_id': '37769a4e-fd8d-4193-8a91-f001a16649b2'},
  {'content': '<b> Такой бэк довольно большой построил с фронтом. Так, что надо сделать?  Да, вот этот бэк, который у меня есть, надо его опубликовать.  Сейчас на NGX поброшу какой-нибудь отдельный.  Так, я не могу зайти на...</b>',
   'speaker': 'Dmitriy Grankin',
   'timestamp': '2024-11-03T12:27:17.964000Z

In [20]:
def test_global_search():

    response = requests.post(
        f"{BASE_URL}/search/global",
        headers=HEADERS,
        json={
            "query": "API discussion",
            "limit": 5,
            "min_score": 0.6
        }
    )
    print("Filtered Global Search:", response.json())

    return response.json()

# Test the function
results = test_global_search()

Filtered Global Search: {'results': [{'topic_name': 'API', 'speaker_name': 'Artem Puzik', 'summary': 'API refers to the application programming interface used for interactions within the booking system.', 'details': 'The speakers discuss the need for various API endpoints to manage bookings, document retrieval, and status updates.', 'meeting_id': '959323cb-74f6-449b-980e-884afe30c7b7', 'timestamp': '2024-10-21T08:31:41.516000', 'score': 0.9728167999999999, 'vector_scores': {'topic_name': 0.9728167999999999, 'summary': 0.5468376899999999}, 'exact_matches': [], 'source': 'main'}, {'topic_name': 'API', 'speaker_name': 'Sergey Ryabenko', 'summary': 'A technical term discussed in relation to document management and templates.', 'details': 'The speakers emphasize the importance of using APIs for dynamic document retrieval and management in their project.', 'meeting_id': '6a43b30f-cac1-4d3e-8d1c-c0346dd02744', 'timestamp': '2024-10-29T12:34:22.180000', 'score': 0.9728167999999999, 'vector_sco

In [21]:
results

{'results': [{'topic_name': 'API',
   'speaker_name': 'Artem Puzik',
   'summary': 'API refers to the application programming interface used for interactions within the booking system.',
   'details': 'The speakers discuss the need for various API endpoints to manage bookings, document retrieval, and status updates.',
   'meeting_id': '959323cb-74f6-449b-980e-884afe30c7b7',
   'timestamp': '2024-10-21T08:31:41.516000',
   'score': 0.9728167999999999,
   'vector_scores': {'topic_name': 0.9728167999999999,
    'summary': 0.5468376899999999},
   'exact_matches': [],
   'source': 'main'},
  {'topic_name': 'API',
   'speaker_name': 'Sergey Ryabenko',
   'summary': 'A technical term discussed in relation to document management and templates.',
   'details': 'The speakers emphasize the importance of using APIs for dynamic document retrieval and management in their project.',
   'meeting_id': '6a43b30f-cac1-4d3e-8d1c-c0346dd02744',
   'timestamp': '2024-10-29T12:34:22.180000',
   'score': 0.97